In [ ]:
# Import required libraries
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.metrics import (
    confusion_matrix, accuracy_score,
    precision_recall_fscore_support, classification_report
)
import warnings
import os

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Set default figure background color to white
plt.rcParams.update({"figure.facecolor": "white"})


In [ ]:
# Define possible locations for the predictions file
CANDIDATES = [
    Path("/content/predictions.csv"),
    Path("predictions.csv"),
    Path("artifacts/predictions.csv")
]

# Define directories for storing artifacts and plots
ART = Path("artifacts")
CM_DIR = ART / "confusion_matrices"
PLOT_DIR = ART / "plots"

# Create directories if they do not exist
ART.mkdir(exist_ok=True)
CM_DIR.mkdir(parents=True, exist_ok=True)
PLOT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# Set layout parameters for confusion matrices and misclassified samples
CM_COLS = 3            # Number of confusion matrix columns per row
CM_SUBPLOT_W = 5       # Width (in inches) of each confusion matrix subplot
CM_SUBPLOT_H = 4       # Height (in inches) of each confusion matrix subplot
MISCLASS_SAMPLES = 5   # Number of misclassified samples to display per model


In [ ]:
# Attempt to find the predictions.csv file from the candidate paths
PRED = next((p for p in CANDIDATES if p.exists()), None)

# Raise an error if the file is not found
if PRED is None:
    print("Could not find predictions.csv. Checked:", [str(p) for p in CANDIDATES])
    raise FileNotFoundError("predictions.csv not found. Please ensure it exists in one of the listed directories.")

print("Using predictions file:", PRED)


In [ ]:
# Load predictions data into a pandas DataFrame
df = pd.read_csv(PRED)

# Define the required columns that must be present in the CSV
required = {"model", "y_true", "y_pred"}
if not required.issubset(df.columns):
    raise ValueError(f"predictions.csv must contain columns {required}. Found: {set(df.columns)}")

# Convert target and prediction columns to string type
df["y_true"] = df["y_true"].astype(str)
df["y_pred"] = df["y_pred"].astype(str)

# Handle missing values by filling them with 'Unknown'
if df.isnull().any().any():
    print("Warning: Missing values detected. Filling missing entries with 'Unknown'.")
    df = df.fillna("Unknown")

# Get unique label names and models from the dataset
labels_order = sorted(df["y_true"].unique().tolist())
models = sorted(df["model"].unique().tolist())

# Print the discovered labels and models
print(f"Detected labels: {labels_order}")
print(f"Detected models: {models}\nTotal examples: {len(df)}")


In [ ]:
# Initialize a list to store model performance results
results = []

# Loop through each model and compute performance metrics
for model_name, grp in df.groupby("model"):
    y_true = grp["y_true"].tolist()
    y_pred = grp["y_pred"].tolist()
    
    # Compute standard classification metrics
    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="weighted", zero_division=0, labels=labels_order
    )
    
    # Append results to list
    results.append({
        "model": model_name,
        "accuracy": float(acc),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1)
    })
    
    # Compute and normalize confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels_order)
    cm_norm = cm.astype(float) / (cm.sum(axis=1, keepdims=True) + 1e-12)
    
    # Plot and save confusion matrix heatmap
    fig, ax = plt.subplots(figsize=(CM_SUBPLOT_W, CM_SUBPLOT_H))
    sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=labels_order, yticklabels=labels_order, ax=ax, cbar=False)
    ax.set_title(f"{model_name} - Normalized Confusion Matrix")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    # Save confusion matrix as PNG
    outpng = CM_DIR / f"{model_name}_confusion_norm.png"
    fig.savefig(outpng, dpi=150)
    plt.close(fig)
    
    # Save classification report as JSON
    crep = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    rep_path = ART / f"{model_name}_classification_report.json"
    with open(rep_path, "w", encoding="utf-8") as fh:
        json.dump(crep, fh, indent=2)
    
    print(f"Saved confusion matrix: {outpng}")
    print(f"Saved classification report: {rep_path}")


In [ ]:
# Create a summary DataFrame from all computed model metrics
summary_df = pd.DataFrame(results).sort_values("f1", ascending=False).reset_index(drop=True)

# Save model comparison results to CSV
summary_csv = ART / "model_comparison.csv"
summary_df.to_csv(summary_csv, index=False)
print(f"Saved model comparison CSV -> {summary_csv}")

# Plot bar chart comparing model performance metrics
if not summary_df.empty:
    metrics = ["accuracy", "precision", "recall", "f1"]
    ax = summary_df.set_index("model")[metrics].plot(kind="bar", figsize=(10,5), rot=45)
    ax.set_ylim(0, 1)
    ax.set_ylabel("Score")
    ax.set_title("Model Comparison (Accuracy, Precision, Recall, F1)")
    
    # Annotate bars with numerical values
    for container in ax.containers:
        ax.bar_label(container, fmt="%.2f", label_type="edge")
    
    plt.tight_layout()
    outplot = PLOT_DIR / "model_comparison_bar.png"
    fig = ax.get_figure()
    fig.savefig(outplot, dpi=150)
    plt.show()
    plt.close(fig)
    print(f"Saved comparison bar chart -> {outplot}")
else:
    print("No models to plot in summary.")
